In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras as k
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [15]:
# Set the path to the Food-101 dataset
rootfolder = '/kaggle/input/indonesian-food/indonesian-food/'
sampleimgpath = '/kaggle/input/indonesian-food/indonesian_food/food/'
sampleimages = ['ayam_bakar/ayam_bakar_(100).jpg', 'gado_gado/gado_gado_(102).jpg', 'rendang/rendang_(104).jpg', 'bakso/bakso_(101).jpg', 'sate/sate_(104).jpg']

# configurations
epoch_count = 100
img_size = (224,224)
img_input_shape = (224,224, 3)  # can be tried with lower size to train model fast
batch_size = 32

# Set the paths for the data directory and the saved model
data_dir = '/kaggle/input/indonesian-food/indonesian_food/food/'
saved_model_path = 'vgg16_model_ver1.h5'

In [16]:
# Limiting as of now for five kinds of foods
# Get the list of folders to use
use_folders = ['ayam_bakar', 'bakso', 'gado_gado', 'rendang', 'sate']

# Get the list of subfolders in the dataset directory
subfolders = os.listdir(data_dir)

# Create a list of the subfolders to use
subfolders_to_use = [subfolder for subfolder in subfolders if subfolder in use_folders]

# Uncomment following line if want to train the model for all 101 foods
# subfolders_to_use = subfolders

# Define the number of classes
num_classes = len(subfolders_to_use)

# Get the list of class names
class_names = sorted(subfolders_to_use)

In [17]:
# Define the image generator for training data and validation data (20%)
# Also apply multiple augmentations
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)


# Define the training and validation generators
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    classes=subfolders_to_use,
    class_mode='categorical',
    subset='training')

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    classes=subfolders_to_use,
    class_mode='categorical',
    subset='validation')


Found 1762 images belonging to 5 classes.
Found 439 images belonging to 5 classes.


In [18]:
# Load pre-trained VGG16 model without top layers
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=img_input_shape)

58889256/58889256 [==============================] - 0s 0us/step


In [23]:
# Freeze pre-trained layers
for layer in vgg16_model.layers:
    layer.trainable = False  

# Add custom top layers
x = vgg16_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Combine pre-trained model with custom top layers
model = Model(inputs=vgg16_model.input, outputs=predictions)

In [24]:
# Compile the model with the Adam optimizer and categorical cross-entropy loss. can be tried with other optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
# Train model
model.fit(train_generator, epochs=epoch_count, validation_data=val_generator)

Epoch 1/100
56/56 [==============================] - 43s 605ms/step - loss: 2.0442 - accuracy: 0.4262 - val_loss: 1.0838 - val_accuracy: 0.5740
Epoch 2/100
56/56 [==============================] - 26s 471ms/step - loss: 0.9305 - accuracy: 0.6493 - val_loss: 0.8163 - val_accuracy: 0.7084
Epoch 3/100
56/56 [==============================] - 27s 486ms/step - loss: 0.7722 - accuracy: 0.7264 - val_loss: 0.7743 - val_accuracy: 0.7084
Epoch 4/100
56/56 [==============================] - 27s 479ms/step - loss: 0.6799 - accuracy: 0.7605 - val_loss: 0.7195 - val_accuracy: 0.7380
Epoch 5/100
56/56 [==============================] - 27s 476ms/step - loss: 0.6264 - accuracy: 0.7628 - val_loss: 0.6892 - val_accuracy: 0.7380
Epoch 6/100
56/56 [==============================] - 27s 474ms/step - loss: 0.5763 - accuracy: 0.7906 - val_loss: 0.7086 - val_accuracy: 0.7449
Epoch 7/100
56/56 [==============================] - 27s 480ms/step - loss: 0.5602 - accuracy: 0.7883 - val_loss: 0.6450 - val_accuracy:

In [26]:
# Evaluate the model on the validation set
score = model.evaluate(val_generator, verbose=0)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])
print('-----------------------------------\n')

# Save the trained model
model.save(saved_model_path)

Validation loss: 0.7086359858512878
Validation accuracy: 0.7949886322021484
-----------------------------------



In [27]:
def GetFoodName(imgpath):
    # Load the image
    img = tf.keras.preprocessing.image.load_img(imgpath, target_size=img_size)

    # Convert the image to an array
    img_array = tf.keras.preprocessing.image.img_to_array(img)

    # Reshape the array to match the input shape of the model
    img_array = np.expand_dims(img_array, axis=0)

    # Normalize the image data
    img_array = img_array / 255.0

    # Make a prediction using the trained model
    prediction = model.predict(img_array)
    print(prediction)

    # Get the predicted class label
    predicted_class_idx = np.argmax(prediction, axis=-1)[0]
    predicted_class_label = class_names[predicted_class_idx]
    return predicted_class_label

In [28]:
# Reload the trained model
model = k.models.load_model('/kaggle/working/' + saved_model_path)

# Check of sample images and print the predicted class label
for img in sampleimages:
    sampleimage = sampleimgpath + img
    predicted_class_label = GetFoodName(sampleimage)
    print("Predicted class label: ", predicted_class_label)

1/1 [==============================] - 0s 414ms/step
[[9.9477780e-01 1.8710241e-03 1.7682315e-03 4.0649367e-04 1.1763728e-03]]
Predicted class label:  ayam_bakar
1/1 [==============================] - 0s 19ms/step
[[4.6487909e-02 5.3602425e-06 4.6342841e-01 6.2903832e-03 4.8378792e-01]]
Predicted class label:  sate
1/1 [==============================] - 0s 19ms/step
[[4.4595757e-03 1.1932838e-05 6.0713518e-01 3.8698992e-01 1.4034084e-03]]
Predicted class label:  gado_gado
1/1 [==============================] - 0s 19ms/step
[[2.1112783e-06 9.9998415e-01 4.8221318e-07 1.0027032e-05 3.1892337e-06]]
Predicted class label:  bakso
1/1 [==============================] - 0s 19ms/step
[[6.4741880e-01 7.9956482e-04 2.8778737e-02 2.2055185e-04 3.2278237e-01]]
Predicted class label:  ayam_bakar
